In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from thefuzz import fuzz
import time
import json

# get urls via api


In [5]:
def are_strings_similar(str1, str2, threshold=70):
    similarity_score = fuzz.ratio(str1, str2)
    return similarity_score >= threshold

In [10]:
# Set your Musixmatch API key here
api_key = "ce31eeb7ac2d7ce756b0e162482a10cf"
api_key2 = "dce2533618a0783cac073f0bda9730a7"

# Set up the API endpoint for track search
search_endpoint = "https://api.musixmatch.com/ws/1.1/track.search"

# Set up the API endpoint for track lyrics
lyrics_endpoint = "https://api.musixmatch.com/ws/1.1/track.lyrics.get"

# Set up the API endpoint for track info via isrc
isrc_endpoint = "https://api.musixmatch.com/ws/1.1/track.get"

In [7]:
# Define a function to fetch lyrics by track_id
def get_lyrics(track_id):
    params = {
        "track_id": track_id,
        "apikey": api_key
    }
    response = requests.get(lyrics_endpoint, params=params)
    if response.status_code == 200:
        lyrics_data = response.json()
        if "message" in lyrics_data and "body" in lyrics_data["message"] and "lyrics" in lyrics_data["message"]["body"]:
            return lyrics_data["message"]["body"]["lyrics"]["lyrics_body"]
    return None

In [8]:
# Define a function to fetch tracks by track name
def get_track_url(track_name, artist_name):
    #only take the first artist
    artist_names = artist_name.split(",")

    params = {
        "q_track": track_name,
        "q_artist": artist_names[0],
        "apikey": api_key,
        "page_size": 100,
        "s_track_rating": "desc"
    }
    response = requests.get(search_endpoint, params=params)
    if response.status_code == 200:
        track_url = None
        search_data = response.json()
        #print search_data with json.dumps
        #print(json.dumps(search_data, indent=4, sort_keys=True))
        if "message" in search_data and "body" in search_data["message"] and "track_list" in search_data["message"]["body"]:
            #print("first check")
            track_list = search_data["message"]["body"]["track_list"]
            found = False
            for track in track_list:
                for artist_name in artist_names: 
                        if are_strings_similar(track["track"]["artist_name"].lower(), artist_name):
                            #print("second check")
                            if track["track"]["has_lyrics"] == 1:
                                track_url = track["track"]["track_share_url"]
                                found = True    
                            elif track["track"]["instrumental"] == 1:
                                track_url = "Instrumental"
                                found = True
                        if found == True:
                            break
                if found == True:
                    break

                            
        return track_url
    else:
        time.sleep(60)
        return(f'Error: {response.status_code}')
    

In [19]:
def get_url_by_isrc(isrc):

    params = {
        "track_isrc": isrc,
        "apikey": api_key,
    }
    response = requests.get(isrc_endpoint, params=params)
    if response.status_code == 200:
        track_url = None
        search_data = response.json()
        #print search_data with json.dumps
        #print(json.dumps(search_data, indent=4, sort_keys=True))
        found = False
        if "message" in search_data and "body" in search_data["message"] and "track" in search_data["message"]["body"]:
            track = search_data["message"]["body"]["track"]
            if track["has_lyrics"] == 1:
                track_url = track["track_share_url"]  
            elif track["instrumental"] == 1:
                track_url = "Instrumental"                        
        return track_url
    else:
        time.sleep(60)
        return(f'Error: {response.status_code}')

In [9]:
#test the function
print(get_track_url("purple rain", "prince"))


https://www.musixmatch.com/lyrics/Prince/Purple-Rain?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753


In [18]:
get_url_by_isrc('GBAFL8600048')

{
    "message": {
        "body": {
            "track": {
                "album_id": 16627279,
                "album_name": "The Moon and the Melodies",
                "artist_id": 1008,
                "artist_name": "Cocteau Twins",
                "commontrack_id": 22276723,
                "explicit": 0,
                "has_lyrics": 1,
                "has_richsync": 0,
                "has_subtitles": 1,
                "instrumental": 0,
                "num_favourite": 6,
                "primary_genres": {
                    "music_genre_list": [
                        {
                            "music_genre": {
                                "music_genre_id": 10,
                                "music_genre_name": "Singer/Songwriter",
                                "music_genre_name_extended": "Singer/Songwriter",
                                "music_genre_parent_id": 34,
                                "music_genre_vanity": "Singer-Songwriter"
                 

'https://www.musixmatch.com/lyrics/Cocteau-Twins/Sea-Swallow-Me?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753'

In [20]:
#load dataframe with url
df = pd.read_csv('data_cut_missing_lyrics.csv')

In [21]:
cnt = 0
error_cnt = 0
progress = 0
progress_end = len(df)//100
for index, row in df.iterrows():
    cnt += 1
    try:
        df.at[index, 'lyrics_url'] = get_url_by_isrc(row['isrc'])
    except:
        df.at[index, 'lyrics_url'] = 'Error'
        error_cnt += 1
    if cnt == 100:
        progress += 1
        #print error count
        print(f'Error count: {error_cnt} - Progress: {progress}/{progress_end}')
        if error_cnt == cnt:
            break
        else:
            cnt = 0
            error_cnt = 0

Error count: 0 - Progress: 1/86
Error count: 0 - Progress: 2/86
Error count: 0 - Progress: 3/86
Error count: 0 - Progress: 4/86
Error count: 0 - Progress: 5/86
Error count: 0 - Progress: 6/86
Error count: 0 - Progress: 7/86
Error count: 0 - Progress: 8/86
Error count: 0 - Progress: 9/86
Error count: 0 - Progress: 10/86
Error count: 0 - Progress: 11/86
Error count: 0 - Progress: 12/86
Error count: 0 - Progress: 13/86
Error count: 0 - Progress: 14/86
Error count: 0 - Progress: 15/86
Error count: 0 - Progress: 16/86
Error count: 0 - Progress: 17/86
Error count: 0 - Progress: 18/86
Error count: 0 - Progress: 19/86
Error count: 0 - Progress: 20/86
Error count: 0 - Progress: 21/86
Error count: 0 - Progress: 22/86
Error count: 0 - Progress: 23/86
Error count: 0 - Progress: 24/86
Error count: 0 - Progress: 25/86
Error count: 0 - Progress: 26/86
Error count: 0 - Progress: 27/86
Error count: 0 - Progress: 28/86
Error count: 0 - Progress: 29/86
Error count: 0 - Progress: 30/86
Error count: 0 - Pr

In [22]:
#lyrics_url not null
df2 = df[df['lyrics_url'].notnull()]
df2.shape

(2921, 37)

In [37]:
#lyrics_url not null and not instrumental
df3 = df2[df2['lyrics_url'] != 'Instrumental']
df3.shape

(1911, 37)

# get lyrics via web scraping

In [25]:
def get_html_from_url(url):
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
    }
    try:
        # Send an HTTP GET request to the URL
        response = requests.get(url)#, headers=headers)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Get the HTML content
            html = response.text
            return html
        else:
            print("HTTP GET request failed with status code: " + str(response.status_code))
            return f"HTTP GET request failed with status code: {response.status_code}"

    except Exception as e:
        return "An error occurred: " + str(e)

In [26]:
def extract_lyrics_from_genius_html(html):
    try:
        # Parse the HTML content
        soup = BeautifulSoup(html, "html.parser")
        # Find the element that contains the lyrics
        ps = soup.find_all("p", class_="mxm-lyrics__content")
        if ps is None or len(ps) == 0:
            return None
        lyrics = "\n".join([p.get_text() for p in ps])
        return lyrics.strip("\n")
    except Exception as e:
        return "An error occurred: " + str(e)


In [27]:
#test the function
html = get_html_from_url("https://www.musixmatch.com/lyrics/AC-DC/Back-in-Black-Fisk-remix")
#print(html)
print(extract_lyrics_from_genius_html(html))

Back in black
I hit the sack
I′ve been too long I'm glad to be back
Yes I′m let loose
From the noose
That's kept me hanging about
Lookin' at the sky
′Cause it′s gettin' me high
Forget the hearse ′cause I'll never die
I got nine lives
Cat′s eyes
Abusin' every one of them and running wild

′Cause I'm back
Yes I'm back
Well I′m back
Yes I′m back
Well I'm hey hey hey hey
(Hey hey hey hey)
Well I′m back in black
Yes I'm back in black

Back in the back
Of a Cadillac
Number one with a bullet
I′m a power pack
Yes, I'm in a band, with a gang
They′ve got to catch me if they want me to hang
Cause I'm back on the track
And I'm beatin′ the flack
Nobody′s gonna get me on another rap
So look at me now
I'm just makin′ my play
Don't try to push your luck
Just get out of my way

′Cause I'm back
Yes I′m back
Well I'm back
Yes I'm back
Well I′m hey hey hey hey
(Hey hey hey hey)
Well I′m back in black
Yes I'm back in black

Well I′m back, Yes I'm back
Well I′m back, Yes I'm back
Well I′m hey hey hey hey
(H

In [66]:
#100 to 200
df = df5.tail(200)

In [67]:
cnt = 0
error_cnt = 0
progress = 0
progress_end = len(df)//100
for index, row in df.iterrows():
    cnt += 1
    try:
        df.at[index, 'lyrics'] = extract_lyrics_from_genius_html(get_html_from_url(row["lyrics_url"]))
    except:
        df.at[index, 'lyrics'] = 'Error'
        error_cnt += 1
    if cnt == 100:
        progress += 1
        #print error count
        print(f'Error count: {error_cnt} - Progress: {progress}/{progress_end}')
        if error_cnt == cnt:
            break
        else:
            cnt = 0
            error_cnt = 0

Error count: 0 - Progress: 1/2
Error count: 0 - Progress: 2/2


In [68]:
#lyrics not null
df[df['lyrics'].notnull()]

,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,chart_power,...,Listeners,Playcount,Tags,clear_name,lyrics_url,lyrics,chart_peak,chart_weeks,rank,chart_rank
4881,GBASQ8400294,"['pop', 'power-pop', 'rock']",worthless thing,elvis costello & the attractions,Goodbye Cruel World,1984,year,spotify:track:7t5C0S2lR2zi4BpAvQcyjR,7t5C0S2lR2zi4BpAvQcyjR,NaN,...,3350,8502,"['indie', '80s', 'new wave', 'Norderdorfer', 'Norderdorfer-Indie']",worthless thing,https://www.musixmatch.com/lyrics/Elvis-Costello-The-Attractions/Worthless-Thing?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753,How many times can you jump out of the cupboard\nBefore someone gets suspicious or someone gets discovered\nYou can live forever in a split second of fame\nCome on down the price is right what′s y...,0,0,0.037532,0
5643,GBBBN8500042,['indie'],platform one,king,Bitter Sweet,1985,year,spotify:track:34VCLXV6VwJjIn99oObXHq,34VCLXV6VwJjIn99oObXHq,NaN,...,546,1497,"['Highly recommended', 'Fantastic Pop Album']",platform one,https://www.musixmatch.com/lyrics/King/Platform-One?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753,放不下 誰在尷尬\n而我 自問自答 練習牽掛\n放不下 誰在害怕\n我塗改了想要說的話\n有些曖昧 你答應約會\n拒絕了誰 你答應約會\n感覺對了 你答應約會\n你答應約會 你答應約會\n\n오빠 強勢 一點 去改變\n改變 別怕 拒絕的 危險\n危險 也要 勇敢 送信件\n信件 像 男人般 握住拳\n\n難道 開開心心 打打鬧鬧 哈哈笑笑 要\nDancers 蹦蹦跳跳 說說聊聊 砰砰心...,0,0,0.030064,0
5647,uscgh0952946,['rock'],drawn to the light,paul clark,Drawn To The Light,1982,year,spotify:track:1zL1o0VE21JSrGabxdzSdU,1zL1o0VE21JSrGabxdzSdU,NaN,...,582,1019,[],drawn to the light,https://www.musixmatch.com/lyrics/Paul-Clark-2/Drawn-to-the-Light?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753,"Oh, Lord have mercy on my soul\nDon′t let the devil take control\nRemove this darkness\nFor eternity\nMake me a soldier of Your army\n\nWhen I am walkin' down the road\nMy legs get so heavy from t...",0,0,0.030061,0
5651,ITB008270804,['pop'],canta sempre,donatella rettore,Kamikaze Rock 'n' Roll Suicide,1982-09-20,day,spotify:track:3rLv8qJVibeXKfdDsOUbTz,3rLv8qJVibeXKfdDsOUbTz,NaN,...,543,1171,[],canta sempre,https://www.musixmatch.com/lyrics/Donatella-Rettore/Canta-sempre?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753,Canta canta canta sempre\nMentre canta sbatte gli occhi\nE mentre canta canta canta sempre\nSembra proprio deficiente.\n\nCanta canta canta sempre\nLe sa tutte vecchie e nuove\nE canta canta canta...,0,0,0.030050,0


In [29]:
#store dataframe
df3.to_csv('lyrics5.csv', index=False)

In [30]:
df3.head()

,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,chart_power,...,Listeners,Playcount,Tags,clear_name,lyrics_url,lyrics,chart_peak,chart_weeks,rank,chart_rank
0,GBAFL8600048,"['alt-rock', 'alternative', 'indie', 'indie-po...","sea, swallow me","cocteau twins,harold budd",The Moon and the Melodies,1986-11-10,day,spotify:track:7vPHiwgCOAjSBRjAirbMvS,7vPHiwgCOAjSBRjAirbMvS,NaN,...,457980,5130122,"['dream pop', 'shoegaze', 'alternative rock', ...","sea, swallow me",None,no lyrics,0,0,0.800000,0
1,GBAFL8400054,"['alt-rock', 'alternative', 'indie', 'indie-po...",lorelei,cocteau twins,Treasure,1984-11-01,day,spotify:track:1ZnXuijpwEBjhNxpEnb8WH,1ZnXuijpwEBjhNxpEnb8WH,NaN,...,380071,2646044,"['dream pop', 'ethereal', '80s', 'alternative'...",lorelei,None,no lyrics,0,0,0.585526,0
4,GBCCS1500052,['rock'],say say say - remastered 2015,"paul mccartney,michael jackson",Pipes Of Peace,1983-10-28,day,spotify:track:1db0gbTSKdRQum8VlbNkiO,1db0gbTSKdRQum8VlbNkiO,NaN,...,110289,490433,"['michael jackson', 'funk pop', 'post-disco', ...",say say say,"Say, say, say what you want\nBut don′t play ga...",no lyrics,0,0,0.425423,0
5,GBAFL8400059,"['alt-rock', 'alternative', 'indie', 'indie-po...",persephone,cocteau twins,Treasure,1984-11-01,day,spotify:track:05qq2YTWLzDSiIUiiKozWh,05qq2YTWLzDSiIUiiKozWh,NaN,...,199585,1144712,"['dream pop', '80s', 'ethereal', 'female vocal...",persephone,None,no lyrics,0,0,0.406357,0
8,GBAFL8300054,"['alt-rock', 'alternative', 'indie', 'indie-po...",sugar hiccup,cocteau twins,Head Over Heels,1983-10-31,day,spotify:track:28gz2W2KMsn1QnvojoWPqv,28gz2W2KMsn1QnvojoWPqv,NaN,...,176144,1041898,"['shoegaze', 'ethereal', 'dream pop', 'post-pu...",sugar hiccup,None,no lyrics,0,0,0.386139,0


In [32]:
#lyrics_url not null
df4 = df3[df3['lyrics_url'].notnull()]
df4.shape

(118, 37)

In [33]:
#lyrics_url not instrumental
df4 = df4[df4['lyrics_url'] != 'Instrumental']
df4.shape

(118, 37)

In [34]:
df4

,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,chart_power,...,Listeners,Playcount,Tags,clear_name,lyrics_url,lyrics,chart_peak,chart_weeks,rank,chart_rank
4,GBCCS1500052,['rock'],say say say - remastered 2015,"paul mccartney,michael jackson",Pipes Of Peace,1983-10-28,day,spotify:track:1db0gbTSKdRQum8VlbNkiO,1db0gbTSKdRQum8VlbNkiO,NaN,...,110289,490433,"['michael jackson', 'funk pop', 'post-disco', ...",say say say,"Say, say, say what you want\nBut don′t play ga...",no lyrics,0,0,0.425423,0
13,GBCCS8300875,['rock'],say say say,"paul mccartney,michael jackson",All The Best (UK Version),1987-11-02,day,spotify:track:2iwhqrveOYC8XLB7GZm63G,2iwhqrveOYC8XLB7GZm63G,NaN,...,96995,450773,"['80s', 'pop', 'duet', 'rock', 'michael jackson']",say say say,"Say, say, say what you want\nBut don′t play ga...",no lyrics,0,0,0.358368,0
16,USWB11506585,"['country', 'pop']",wildflowers - 2015 remaster,"dolly parton,linda ronstadt,emmylou harris",Trio (2016 Remaster),1987-03-03,day,spotify:track:27876bQcDNkZhWqvF0zfFN,27876bQcDNkZhWqvF0zfFN,NaN,...,40887,134866,[],wildflowers,The hills were alive with wildflowers\nAnd I w...,no lyrics,0,0,0.339453,0
19,GBAKW8801005,['punk'],peek-a-boo,siouxsie and the banshees,Peepshow (Remastered And Expanded),1988-09-05,day,spotify:track:3VkyKA0iCT1ABwaSjb0kpf,3VkyKA0iCT1ABwaSjb0kpf,NaN,...,147371,656707,"['80s', 'post-punk', 'new wave', 'goth', 'alte...",peek,Creeping up the backstairs\nSlinking into dark...,no lyrics,0,0,0.328701,0
24,USRH10175221,"['pop', 'r-n-b', 'rock', 'rock-n-roll', 'rocka...","december, 1963 (oh what a night!)",frankie valli & the four seasons,Anthology,1988-09-27,day,spotify:track:11rNF45OFPZczULtwIKxC9,11rNF45OFPZczULtwIKxC9,NaN,...,279416,1575208,"['oldies', 'Disco', 'pop', '70s', 'funk']","december, 1963","Oh, what a night\nLate December back in sixty ...",no lyrics,0,0,0.320700,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,SELAA0503711,"['pop', 'punk', 'swedish']",hey winner,lolita pop,Love Poison,1989-01-01,day,spotify:track:2NeodZQ15mlBkOUi3SspQ6,2NeodZQ15mlBkOUi3SspQ6,NaN,...,3933,14110,"['80s', 'swedish', 'pop', 'rock', 'three']",hey winner,"Hey there, winner\nWon′t you please come down\...",no lyrics,0,0,0.187259,0
349,GBF068025060,"['alt-rock', 'alternative', 'pop', 'punk', 'pu...",clockface,siouxsie and the banshees,Kaleidoscope (Remastered & Expanded),1980-08-01,day,spotify:track:3lbdPjt4RXfaIgS2Npowv5,3lbdPjt4RXfaIgS2Npowv5,NaN,...,45038,154107,"['post-punk', 'new wave', '80s', 'goth', 'Goth...",clockface,"Ohh, ohh, ohh, ohh, ohh, ohh\n\nWhoa-oh whoa-o...",no lyrics,0,0,0.186792,0
602,USSM18400418,"['blues', 'rock']",tin pan alley (aka roughest place in town),stevie ray vaughan,Couldn't Stand The Weather,1984,year,spotify:track:2dKDzmOCLox95RF9xXA02J,2dKDzmOCLox95RF9xXA02J,NaN,...,63339,239106,"['blues', 'blues rock', 'guitar virtuoso', 'el...",tin pan alley,Went down to Tin Pan Alley\nTo see what was go...,no lyrics,0,0,0.162279,0
1252,USCA21202400,"['alt-rock', 'alternative', 'indie', 'rock']",strange - remastered,r.e.m.,Document - 25th Anniversary Edition,1987-09-01,day,spotify:track:4xP8dSzmS3FxyjL1FHAHyG,4xP8dSzmS3FxyjL1FHAHyG,NaN,...,2674,5523,[],strange,There′s something strange going on tonight\nTh...,no lyrics,0,0,0.120703,0


In [35]:
#put lyrics_url in lyrics column
df4['lyrics'] = df4['lyrics_url']

In [36]:
#add lyrics_url from df2 to df4 compared by uri
for index, row in df4.iterrows():
    df4.at[index, 'lyrics'] = df2[df2['uri'] == row['uri']]['lyrics_url'].values[0]
df4

,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,chart_power,...,Listeners,Playcount,Tags,clear_name,lyrics_url,lyrics,chart_peak,chart_weeks,rank,chart_rank
4,GBCCS1500052,['rock'],say say say - remastered 2015,"paul mccartney,michael jackson",Pipes Of Peace,1983-10-28,day,spotify:track:1db0gbTSKdRQum8VlbNkiO,1db0gbTSKdRQum8VlbNkiO,NaN,...,110289,490433,"['michael jackson', 'funk pop', 'post-disco', ...",say say say,"Say, say, say what you want\nBut don′t play ga...",https://www.musixmatch.com/lyrics/Paul-McCartn...,0,0,0.425423,0
13,GBCCS8300875,['rock'],say say say,"paul mccartney,michael jackson",All The Best (UK Version),1987-11-02,day,spotify:track:2iwhqrveOYC8XLB7GZm63G,2iwhqrveOYC8XLB7GZm63G,NaN,...,96995,450773,"['80s', 'pop', 'duet', 'rock', 'michael jackson']",say say say,"Say, say, say what you want\nBut don′t play ga...",https://www.musixmatch.com/lyrics/33191768/113...,0,0,0.358368,0
16,USWB11506585,"['country', 'pop']",wildflowers - 2015 remaster,"dolly parton,linda ronstadt,emmylou harris",Trio (2016 Remaster),1987-03-03,day,spotify:track:27876bQcDNkZhWqvF0zfFN,27876bQcDNkZhWqvF0zfFN,NaN,...,40887,134866,[],wildflowers,The hills were alive with wildflowers\nAnd I w...,https://www.musixmatch.com/lyrics/Dolly-Parton...,0,0,0.339453,0
19,GBAKW8801005,['punk'],peek-a-boo,siouxsie and the banshees,Peepshow (Remastered And Expanded),1988-09-05,day,spotify:track:3VkyKA0iCT1ABwaSjb0kpf,3VkyKA0iCT1ABwaSjb0kpf,NaN,...,147371,656707,"['80s', 'post-punk', 'new wave', 'goth', 'alte...",peek,Creeping up the backstairs\nSlinking into dark...,https://www.musixmatch.com/lyrics/Siouxsie-fea...,0,0,0.328701,0
24,USRH10175221,"['pop', 'r-n-b', 'rock', 'rock-n-roll', 'rocka...","december, 1963 (oh what a night!)",frankie valli & the four seasons,Anthology,1988-09-27,day,spotify:track:11rNF45OFPZczULtwIKxC9,11rNF45OFPZczULtwIKxC9,NaN,...,279416,1575208,"['oldies', 'Disco', 'pop', '70s', 'funk']","december, 1963","Oh, what a night\nLate December back in sixty ...",https://www.musixmatch.com/lyrics/The-Four-Sea...,0,0,0.320700,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,SELAA0503711,"['pop', 'punk', 'swedish']",hey winner,lolita pop,Love Poison,1989-01-01,day,spotify:track:2NeodZQ15mlBkOUi3SspQ6,2NeodZQ15mlBkOUi3SspQ6,NaN,...,3933,14110,"['80s', 'swedish', 'pop', 'rock', 'three']",hey winner,"Hey there, winner\nWon′t you please come down\...",https://www.musixmatch.com/lyrics/Lolita-Pop/H...,0,0,0.187259,0
349,GBF068025060,"['alt-rock', 'alternative', 'pop', 'punk', 'pu...",clockface,siouxsie and the banshees,Kaleidoscope (Remastered & Expanded),1980-08-01,day,spotify:track:3lbdPjt4RXfaIgS2Npowv5,3lbdPjt4RXfaIgS2Npowv5,NaN,...,45038,154107,"['post-punk', 'new wave', '80s', 'goth', 'Goth...",clockface,"Ohh, ohh, ohh, ohh, ohh, ohh\n\nWhoa-oh whoa-o...",https://www.musixmatch.com/lyrics/Siouxsie-and...,0,0,0.186792,0
602,USSM18400418,"['blues', 'rock']",tin pan alley (aka roughest place in town),stevie ray vaughan,Couldn't Stand The Weather,1984,year,spotify:track:2dKDzmOCLox95RF9xXA02J,2dKDzmOCLox95RF9xXA02J,NaN,...,63339,239106,"['blues', 'blues rock', 'guitar virtuoso', 'el...",tin pan alley,Went down to Tin Pan Alley\nTo see what was go...,https://www.musixmatch.com/lyrics/Stevie-Ray-V...,0,0,0.162279,0
1252,USCA21202400,"['alt-rock', 'alternative', 'indie', 'rock']",strange - remastered,r.e.m.,Document - 25th Anniversary Edition,1987-09-01,day,spotify:track:4xP8dSzmS3FxyjL1FHAHyG,4xP8dSzmS3FxyjL1FHAHyG,NaN,...,2674,5523,[],strange,There′s something strange going on tonight\nTh...,https://www.musixmatch.com/lyrics/R-E-M/Strang...,0,0,0.120703,0


In [ ]:
df4.to_csv('lyrics6.csv', index=False)

In [39]:
#drop songs from df3 that are in df4
df5 = df3[~df3['uri'].isin(df4['uri'])]

In [45]:
df5.shape

(1793, 37)

In [44]:
pd.options.display.max_colwidth = 200
df5[['name', 'artists', 'lyrics_url']].sample(10)

,name,artists,lyrics_url
1971,until i believe in my soul,dexys midnight runners,https://www.musixmatch.com/lyrics/Dexys-Midnight-Runners/Until-I-Believe-in-My-Soul?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753
5625,geronimo,raging slab,https://www.musixmatch.com/lyrics/Raging-Slab/Geronimo?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753
1346,on and on,raven,https://www.musixmatch.com/lyrics/Raven/On-and-On?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753
2010,men called uncle,elvis costello & the attractions,https://www.musixmatch.com/lyrics/Elvis-Costello-The-Attractions/Men-Called-Uncle?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753
1238,power of suggestion,steve stevens,https://www.musixmatch.com/lyrics/Steve-Stevens/Power-of-Suggestion?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753
398,danger calling,icon,https://www.musixmatch.com/lyrics/Icon-9/Danger-Calling?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753
5352,athletic rock,raven,https://www.musixmatch.com/lyrics/Raven/Athletic-Rock?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753
2800,black rat,big mama thornton,https://www.musixmatch.com/lyrics/Big-Mama-Thornton/Black-Rat?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753
4881,worthless thing,elvis costello & the attractions,https://www.musixmatch.com/lyrics/Elvis-Costello-The-Attractions/Worthless-Thing?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753
2394,blau auf'm bau,dimple minds,https://www.musixmatch.com/lyrics/Dimple-Minds/Blau-auf-m-Bau?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753


In [69]:
pd.options.display.max_colwidth = 200
df.tail(5)

,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,chart_power,...,Listeners,Playcount,Tags,clear_name,lyrics_url,lyrics,chart_peak,chart_weeks,rank,chart_rank
5642,SEVFZ1104019,"['pop', 'swedish']",de tusen sjöars land,ray adams,60-Tals Pop Jubileum - Live At Tyrol,1989-05-12,day,spotify:track:05MrqdvSPnQ5FgGAeQHuNb,05MrqdvSPnQ5FgGAeQHuNb,NaN,...,562,1321,['swedish'],de tusen sjöars land,https://www.musixmatch.com/lyrics/Ray-Adams/De-tusen-sj%C3%B6ars-land?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753,None,0,0,0.030064,0
5643,GBBBN8500042,['indie'],platform one,king,Bitter Sweet,1985,year,spotify:track:34VCLXV6VwJjIn99oObXHq,34VCLXV6VwJjIn99oObXHq,NaN,...,546,1497,"['Highly recommended', 'Fantastic Pop Album']",platform one,https://www.musixmatch.com/lyrics/King/Platform-One?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753,放不下 誰在尷尬\n而我 自問自答 練習牽掛\n放不下 誰在害怕\n我塗改了想要說的話\n有些曖昧 你答應約會\n拒絕了誰 你答應約會\n感覺對了 你答應約會\n你答應約會 你答應約會\n\n오빠 強勢 一點 去改變\n改變 別怕 拒絕的 危險\n危險 也要 勇敢 送信件\n信件 像 男人般 握住拳\n\n難道 開開心心 打打鬧鬧 哈哈笑笑 要\nDancers 蹦蹦跳跳 說說聊聊 砰砰心...,0,0,0.030064,0
5647,uscgh0952946,['rock'],drawn to the light,paul clark,Drawn To The Light,1982,year,spotify:track:1zL1o0VE21JSrGabxdzSdU,1zL1o0VE21JSrGabxdzSdU,NaN,...,582,1019,[],drawn to the light,https://www.musixmatch.com/lyrics/Paul-Clark-2/Drawn-to-the-Light?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753,"Oh, Lord have mercy on my soul\nDon′t let the devil take control\nRemove this darkness\nFor eternity\nMake me a soldier of Your army\n\nWhen I am walkin' down the road\nMy legs get so heavy from t...",0,0,0.030061,0
5649,USWB10200626,"['country', 'rock', 'singer-songwriter', 'songwriter']",say you will,jd souther,Home By Dawn (Expanded Edition),1985,year,spotify:track:0uY60ikRI1rFSchuO2GeQA,0uY60ikRI1rFSchuO2GeQA,NaN,...,489,2025,['west coast'],say you will,https://www.musixmatch.com/lyrics/JD-Souther/Say-You-Will?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753,None,0,0,0.030060,0
5651,ITB008270804,['pop'],canta sempre,donatella rettore,Kamikaze Rock 'n' Roll Suicide,1982-09-20,day,spotify:track:3rLv8qJVibeXKfdDsOUbTz,3rLv8qJVibeXKfdDsOUbTz,NaN,...,543,1171,[],canta sempre,https://www.musixmatch.com/lyrics/Donatella-Rettore/Canta-sempre?utm_source=application&utm_campaign=api&utm_medium=Fh-Kiel%3A1409623865753,Canta canta canta sempre\nMentre canta sbatte gli occhi\nE mentre canta canta canta sempre\nSembra proprio deficiente.\n\nCanta canta canta sempre\nLe sa tutte vecchie e nuove\nE canta canta canta...,0,0,0.030050,0
